# Functional Gaussian process example

First, import the required libraries

In [ ]:
# Standard imports
import os
from pprint import pprint

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Project imports
import twinlab as tl

Now, define some parameters

In [ ]:
# File paths
campaign_dir = os.path.join("resources", "campaigns", "ukaea")
datasets_dir = os.path.join("resources", "datasets")
filepath_grid = os.path.join(campaign_dir, "grid.csv")
filepath_train = os.path.join(datasets_dir, "ukaea_small.csv")
# filepath_eval = os.path.join(campaign_dir, "eval.csv")
# filepath_eval = os.path.join(campaign_dir, "post.csv")
filepath_eval = os.path.join(campaign_dir, "test.csv")

# Campaign parameters
campaign_id = "ukaea"
dataset_id = "ukaea"

Load the necessary data for training and plotting

In [ ]:
df_train = pd.read_csv(filepath_train)
df_eval = pd.read_csv(filepath_eval)
df_grid = pd.read_csv(filepath_grid, header=None)

Next, we create the parameter dictionary that we need to give to run twinLab.  At a minimum the user must provide the `filename` of the dataset on which we want to train our model (csv format), together with the columns that we will take to be the `inputs` and `outputs` of our model, once that is trained. By default, `twinLab` will train a Gaussian process (`emulator=gaussian_process`) and use all of the data for training. This latter choice can be overridden by adding e.g., `train_test_split=100` so that only the first 100 entries in `filename` are used for training, and the remaining examples can then be used for model evaluation.

In [ ]:
# Column headings for outputs
y_outputs = [f"y{i}" for i in range(len(df_grid))]

# Parameters
params = {
    "dataset_id": dataset_id,
    "inputs": ["E1", "E2", "E3", "n1", "n2"],
    "outputs": y_outputs,
    "decompose_outputs": True,
    "test_train_ratio": 0.8,
}
pprint(params, compact=True)

Now the dataset can be uploaded to the cloud

In [ ]:
tl.upload_dataset(filepath_train, dataset_id, verbose=True)

List the datasets to check that the upload worked

In [ ]:
_ = tl.list_datasets(verbose=True)

Print some useful properties of the dataset

In [ ]:
_ = tl.query_dataset(dataset_id, verbose=True)

Train the model, this step should take less than two minutes.

In [ ]:
tl.train_campaign(params, campaign_id, verbose=True)

List the campaigns again to ensure that training has been completed

In [ ]:
_ = tl.list_campaigns(verbose=True)

Query the campaign to check how training has been

In [ ]:
_ = tl.query_campaign(campaign_id, verbose=True)

Evaluate the trained emulator on `X` from the evaluation file

In [ ]:
df_mean, df_std = tl.predict_campaign(filepath_eval, campaign_id, verbose=False)

Plot the results for a few different `y` values to sanity check

In [ ]:
# Parameters for plot
color = "blue"
alpha = 0.8
xs = {"E1": r"$E_{1}$", "E2": r"$E_{2}$", "E3": r"$E_{3}$", "n1": r"$n_{1}$", "n2": r"$n_{2}$"}
ys = {f"y{i}": fr"$y_{{{i}}}$" for i in [0, 100, 150]}

# Plot some examples
nrow, ncol = len(ys), len(xs)
plt.subplots(nrow, ncol, figsize=(25, 10))
nplot = 0
for y, y_label in ys.items():
    for x, x_label in xs.items():
        nplot += 1
        plt.subplot(nrow, ncol, nplot)
        plt.errorbar(df_eval[x], df_mean[y], yerr=df_std[y], marker='.', lw=1, ls='None', color=color, alpha=alpha, label="Model")
        plt.plot(df_train[x], df_train[y], ".", color="black", alpha=0.1, label="Training data")
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        if nplot==1: plt.legend()
plt.show()

Plot the output of the trained function together with the "truth" from the evaluation file. The intensity of the blue regions here correspond to the probability. It can be seen that the truth mainly goes through the regions of high probability as predicted by the model.

In [ ]:
# Parameters for plot
error_inflation_factor = 1. # Factor to multiply error by for plotting
y_fac = 18 # Factor to divide y by for plotting [log10]
plot_eval = True
data_alpha = 0.75
plot_model_mean = False
plot_model_bands = True
plot_model_blur = False
nsigs = [1, 2]
model_alpha = 0.75
n_model_blur = 100
model_color = "blue"
number_of_training_examples = 0
number_of_model_examples = 5

# Plot results
grid = df_grid.iloc[:, 0]
plt.subplots()
if (plot_model_blur or plot_model_bands) and not plot_model_mean: 
    plt.fill_between(grid, np.nan, np.nan, color=model_color, alpha=model_alpha, lw=0., label="Model predictions")
for example in range(number_of_training_examples): # Training examples
    train = df_train[y_outputs].iloc[example]/10**y_fac
    label = "Example training data" if example==0 else None
    plt.plot(grid, train, color="black", alpha=0.5, label=label)
for example in range(number_of_model_examples): # Model predictions
    mean = df_mean[y_outputs].iloc[example]/10**y_fac
    err = error_inflation_factor*df_std[y_outputs].iloc[example]/10**y_fac
    if plot_eval and (filepath_eval == os.path.join(campaign_dir, "test.csv")):
        eval = df_eval[y_outputs].iloc[example]/10**y_fac
        label = "Test data" if example==0 else None
        plt.plot(grid, eval, color="black", alpha=data_alpha, label=label)
    if plot_model_mean:
        label = "Model predictions" if example==0 else None
        plt.plot(grid, mean, color=model_color, label=label, alpha=model_alpha)
    elif plot_model_bands:
        for nsig in nsigs:
            plt.fill_between(grid, mean-nsig*err, mean+nsig*err, color=model_color, alpha=model_alpha/nsig, lw=0.)
    elif plot_model_blur:
        alpha = tl.get_blur_alpha(n_model_blur, model_alpha)
        dys = tl.get_blur_boundaries(n_model_blur)
        for dy in dys:
            plt.fill_between(grid, mean-dy*err, mean+dy*err, color=model_color, alpha=alpha, lw=0.)
plt.xlabel(r'Temperature [K]')
plt.xlim((grid.min(), grid.max()))
plt.ylabel(rf"Desorption rate [$10^{{{y_fac}}}$ $m^{{{-2}}}$ $s^{{{-1}}}$]")
plt.ylim(bottom=0.)
plt.legend()
plt.show()

Finally, delete the campaign and dataset if necessary

In [ ]:
tl.delete_campaign(campaign_id, verbose=True)
tl.delete_dataset(dataset_id, verbose=True)